In [1]:
import pandas as pd
import math
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, \
    f1_score, roc_auc_score
import catboost
from catboost import CatBoostClassifier, Pool

import warnings
warnings.filterwarnings('ignore')

Задача - определение пола абонента на основании банковских транзакционных данных .  

Для каждого клиента банка из файла test_set.csv проставьте предиктив пола абонента по следующему примеру:   

client_id;gender_prediction  
61240380;1  
34114030;0  
61240380;0  
34114031;1  

Файлы:
types.csv  - справочник типов транзакций  
codes.csv - справочник кодов транзакций  
transactions.csv - транзакционные данные по банковским операциям   
train_set.csv - тренировочный сет с разметкой пола абонента(0/1  - пол абонента)  
test_set.csv - тестовый сет, который необходимо разметить  
  

Описание полей для файла transactions.csv:  
  
client_id - id клиента банка  
datetime - дата транзакции(формат - порядковый_номер_дня час:минута:секунда.пример - 421 06:33:15)  
code - номер кода транзакции  
type - номер типа транзакции  
sum - сумма транзакции  

In [2]:
types = pd.read_csv("C:/Users/Mr0Wo/OneDrive/Рабочий стол/Git_repositories/Data-science-cases/ML-cases/user-gender-detection/types.csv", sep=';')
test_set = pd.read_csv("C:/Users/Mr0Wo/OneDrive/Рабочий стол/Git_repositories/Data-science-cases/ML-cases/user-gender-detection/test_set.csv",sep=';')
codes = pd.read_csv("C:/Users/Mr0Wo/OneDrive/Рабочий стол/Git_repositories/Data-science-cases/ML-cases/user-gender-detection/codes.csv", sep=';')
transactions = pd.read_csv("C:/Users/Mr0Wo/OneDrive/Рабочий стол/Git_repositories/Data-science-cases/ML-cases/user-gender-detection/transactions.csv", sep=';')
train_set = pd.read_csv("C:/Users/Mr0Wo/OneDrive/Рабочий стол/Git_repositories/Data-science-cases/ML-cases/user-gender-detection/train_set.csv", sep=';')

In [3]:
train_set.head()

,client_id,target
0,75063019,0
1,86227647,1
2,6506523,0
3,50615998,0
4,95213230,0


In [4]:
test_set.head()

,client_id
0,61240380
1,34114030
2,15926722
3,50598019
4,76475250


In [5]:
transactions.head()

,client_id,datetime,code,type,sum
0,96372458,421 06:33:15,6011,2010,-561478.94
1,24567813,377 17:20:40,6011,7010,67377.47
2,21717441,55 13:38:47,6011,2010,-44918.32
3,14331004,263 12:57:08,6011,2010,-3368873.66
4,85302434,151 10:34:12,4814,1030,-3368.87


In [6]:
types.head()

,type,type_description
0,8001,Установление расх. лимита по карте
1,2411,Перевод с карты на счет др.лица в одном тер. б...
2,4035,н/д(нет данных)
3,3001,Комиссия за обслуживание ссудного счета
4,2420,Перевод с карты на счет физ.лица в другом тер....


In [7]:
codes.head()

,code,code_description
0,5944,"Магазины по продаже часов, ювелирных изделий и..."
1,5621,Готовые сумочные изделия
2,5697,"Услуги по переделке, починке и пошиву одежды"
3,7995,Транзакции по азартным играм
4,5137,"Мужская, женская и детская спец-одежда"


# WorkFlow


In [8]:
df = transactions.merge(train_set, how='left')

In [9]:
df

,client_id,datetime,code,type,sum,target
0,96372458,421 06:33:15,6011,2010,-561478.94,0.0
1,24567813,377 17:20:40,6011,7010,67377.47,NaN
2,21717441,55 13:38:47,6011,2010,-44918.32,0.0
3,14331004,263 12:57:08,6011,2010,-3368873.66,NaN
4,85302434,151 10:34:12,4814,1030,-3368.87,0.0
...,...,...,...,...,...,...
130034,15836839,147 11:50:53,5411,1010,-26344.59,NaN
130035,28369355,305 11:59:34,4829,2330,-24705.07,0.0
130036,40949707,398 21:13:58,5411,1110,-40353.72,1.0
130037,7174462,409 13:58:14,5411,1010,-25536.06,1.0


In [10]:
ml_df = df.groupby('client_id')['code'].count().reset_index().rename(columns={'code':'transactions_count'})

In [11]:
ml_df

,client_id,transactions_count
0,22899,9
1,27914,4
2,28753,13
3,31385,13
4,38084,26
...,...,...
8651,99967537,1
8652,99984336,4
8653,99985917,1
8654,99991245,4


In [12]:
t = df.groupby('client_id')['code'].value_counts()
t

client_id  code
22899      6011    4
           6010    2
           4814    1
           5399    1
           5541    1
                  ..
99999680   6011    4
           5541    3
           5411    2
           5499    1
           5661    1
Name: code, Length: 46663, dtype: int64

In [13]:
t.name = 'code_count'

In [14]:
t = t.reset_index().sort_values(['client_id', 'code_count'], ascending=False)

In [15]:
t['client_id'].nunique()

8656

In [16]:
t = t.groupby('client_id')['code'].first().reset_index().rename(columns={'code':'code_mode'})

In [17]:
ml_df = ml_df.merge(t, how='left')

In [18]:
ml_df

,client_id,transactions_count,code_mode
0,22899,9,6011
1,27914,4,4814
2,28753,13,4814
3,31385,13,4814
4,38084,26,6011
...,...,...,...
8651,99967537,1,6011
8652,99984336,4,6011
8653,99985917,1,6011
8654,99991245,4,6011


In [19]:
t = df.groupby('client_id')['type'].value_counts()
t.name = 'type_count'
t = t.reset_index().sort_values(['client_id', 'type_count'], ascending=False)
t = t.groupby('client_id')['type'].first().reset_index().rename(columns={'type':'type_mode'})
ml_df = ml_df.merge(t, how='left')

In [20]:
ml_df

,client_id,transactions_count,code_mode,type_mode
0,22899,9,6011,4010
1,27914,4,4814,1030
2,28753,13,4814,1010
3,31385,13,4814,1030
4,38084,26,6011,1010
...,...,...,...,...
8651,99967537,1,6011,2010
8652,99984336,4,6011,7010
8653,99985917,1,6011,2010
8654,99991245,4,6011,7010


In [21]:
t = df.groupby('client_id')['sum'].sum().reset_index()

In [22]:
ml_df = ml_df.merge(t)

In [23]:
ml_df

,client_id,transactions_count,code_mode,type_mode,sum
0,22899,9,6011,4010,50847.54
1,27914,4,4814,1030,74115.21
2,28753,13,4814,1010,-2589800.29
3,31385,13,4814,1030,-83525.38
4,38084,26,6011,1010,693495.66
...,...,...,...,...,...
8651,99967537,1,6011,2010,-336887.37
8652,99984336,4,6011,7010,78607.06
8653,99985917,1,6011,2010,-224591.58
8654,99991245,4,6011,7010,569609.16


## Baseline

In [24]:
train_df = train_set.merge(ml_df)

In [25]:
train_df

,client_id,target,transactions_count,code_mode,type_mode,sum
0,75063019,0,29,6011,1010,89032.60
1,86227647,1,27,6011,1030,-606058.60
2,6506523,0,53,6010,7070,2635753.74
3,50615998,0,7,4814,1030,-42672.40
4,95213230,0,34,4814,1030,214292.66
...,...,...,...,...,...,...
5995,71577803,0,5,7230,1010,-114601.90
5996,8128547,1,7,6011,1030,-262708.36
5997,26055781,0,6,4829,1010,-42863.31
5998,73504380,1,19,5411,1010,-75992.84


In [26]:
X = train_df.drop(columns=['client_id', 'target'])
y = train_df.target

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [28]:
cb_model = CatBoostClassifier(verbose=100)

In [29]:
cb_model.fit(X_train, y_train)

Learning rate set to 0.020129
0:	learn: 0.6922368	total: 140ms	remaining: 2m 19s
100:	learn: 0.6613121	total: 327ms	remaining: 2.91s
200:	learn: 0.6518821	total: 538ms	remaining: 2.14s
300:	learn: 0.6455728	total: 734ms	remaining: 1.7s
400:	learn: 0.6400762	total: 947ms	remaining: 1.42s
500:	learn: 0.6332070	total: 1.14s	remaining: 1.14s
600:	learn: 0.6245969	total: 1.33s	remaining: 882ms
700:	learn: 0.6162253	total: 1.51s	remaining: 646ms
800:	learn: 0.6088395	total: 1.7s	remaining: 421ms
900:	learn: 0.6017292	total: 1.89s	remaining: 208ms
999:	learn: 0.5953749	total: 2.07s	remaining: 0us


In [30]:
y_pred = cb_model.predict(X_test)

In [31]:
f1_score(y_test, y_pred)

0.46650717703349287

In [32]:
roc_auc_score(y_test, y_pred)

0.5944677169266617

## Feature engineering

In [33]:
ml_df

,client_id,transactions_count,code_mode,type_mode,sum
0,22899,9,6011,4010,50847.54
1,27914,4,4814,1030,74115.21
2,28753,13,4814,1010,-2589800.29
3,31385,13,4814,1030,-83525.38
4,38084,26,6011,1010,693495.66
...,...,...,...,...,...
8651,99967537,1,6011,2010,-336887.37
8652,99984336,4,6011,7010,78607.06
8653,99985917,1,6011,2010,-224591.58
8654,99991245,4,6011,7010,569609.16


In [34]:
t = df.groupby('client_id')['type'].value_counts()
t.name = 'type_count'
t = t.reset_index()
t = t.groupby('client_id').head(3)
t = t.groupby('client_id')['type'].sum().reset_index().rename(columns={'type':'sum_type'})
t

,client_id,sum_type
0,22899,6050
1,27914,15060
2,28753,3150
3,31385,3150
4,38084,9230
...,...,...
8651,99967537,2010
8652,99984336,9020
8653,99985917,2010
8654,99991245,10350


In [35]:
ml_df = ml_df.merge(t)
ml_df

,client_id,transactions_count,code_mode,type_mode,sum,sum_type
0,22899,9,6011,4010,50847.54,6050
1,27914,4,4814,1030,74115.21,15060
2,28753,13,4814,1010,-2589800.29,3150
3,31385,13,4814,1030,-83525.38,3150
4,38084,26,6011,1010,693495.66,9230
...,...,...,...,...,...,...
8651,99967537,1,6011,2010,-336887.37,2010
8652,99984336,4,6011,7010,78607.06,9020
8653,99985917,1,6011,2010,-224591.58,2010
8654,99991245,4,6011,7010,569609.16,10350


In [36]:
t = df.groupby('client_id')['code'].value_counts()
t.name = 'code_count'
t = t.reset_index()
t = t.groupby('client_id').head(3)
t = t.groupby('client_id')['code'].sum().reset_index().rename(columns={'code':'sum_code'})
ml_df = ml_df.merge(t)
ml_df

,client_id,transactions_count,code_mode,type_mode,sum,sum_type,sum_code
0,22899,9,6011,4010,50847.54,6050,16835
1,27914,4,4814,1030,74115.21,15060,16835
2,28753,13,4814,1010,-2589800.29,3150,15457
3,31385,13,4814,1030,-83525.38,3150,16236
4,38084,26,6011,1010,693495.66,9230,16251
...,...,...,...,...,...,...,...
8651,99967537,1,6011,2010,-336887.37,2010,6011
8652,99984336,4,6011,7010,78607.06,9020,6011
8653,99985917,1,6011,2010,-224591.58,2010,6011
8654,99991245,4,6011,7010,569609.16,10350,16339


In [37]:
train_df_1 = train_set.merge(ml_df)
X_1 = train_df_1.drop(columns=['client_id', 'target'])
y_1 = train_df_1.target
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=.2, random_state=42)

In [38]:
cb_model_1 = CatBoostClassifier(verbose=100)
cb_model_1.fit(X_train_1, y_train_1)

Learning rate set to 0.020129
0:	learn: 0.6922999	total: 2.12ms	remaining: 2.12s
100:	learn: 0.6573181	total: 198ms	remaining: 1.76s
200:	learn: 0.6434971	total: 390ms	remaining: 1.55s
300:	learn: 0.6326631	total: 598ms	remaining: 1.39s
400:	learn: 0.6231449	total: 814ms	remaining: 1.22s
500:	learn: 0.6123039	total: 1.01s	remaining: 1.01s
600:	learn: 0.6009471	total: 1.21s	remaining: 805ms
700:	learn: 0.5894824	total: 1.41s	remaining: 602ms
800:	learn: 0.5785038	total: 1.62s	remaining: 403ms
900:	learn: 0.5684284	total: 1.83s	remaining: 201ms
999:	learn: 0.5586858	total: 2.05s	remaining: 0us


In [39]:
y_pred_1 = cb_model_1.predict(X_test_1)

In [40]:
f1_score(y_test_1, y_pred_1)

0.47228915662650606

In [41]:
roc_auc_score(y_test_1, y_pred_1)

0.6004728739945003

In [42]:
t = df.groupby('client_id')['code'].value_counts()
t.name = 'code_count'
t = t.reset_index()
t = t.groupby('client_id').head(3)
t = t.groupby('client_id')['code'].apply(list).apply(math.prod).reset_index().rename(columns={'code':'mult_code_top3'})
ml_df = ml_df.merge(t)
ml_df

,client_id,transactions_count,code_mode,type_mode,sum,sum_type,sum_code,mult_code_top3
0,22899,9,6011,4010,50847.54,6050,16835,173911093540
1,27914,4,4814,1030,74115.21,15060,16835,173911093540
2,28753,13,4814,1010,-2589800.29,3150,15457,135156930084
3,31385,13,4814,1030,-83525.38,3150,16236,156577858094
4,38084,26,6011,1010,693495.66,9230,16251,157065740909
...,...,...,...,...,...,...,...,...
8651,99967537,1,6011,2010,-336887.37,2010,6011,6011
8652,99984336,4,6011,7010,78607.06,9020,6011,6011
8653,99985917,1,6011,2010,-224591.58,2010,6011,6011
8654,99991245,4,6011,7010,569609.16,10350,16339,159620127381


In [43]:
t = df.groupby('client_id')['type'].value_counts()
t.name = 'type_count'
t = t.reset_index()
t = t.groupby('client_id').head(3)
t = t.groupby('client_id')['type'].apply(list).apply(math.prod).reset_index().rename(columns={'type':'mult_type_top3'})
ml_df = ml_df.merge(t)
ml_df

,client_id,transactions_count,code_mode,type_mode,sum,sum_type,sum_code,mult_code_top3,mult_type_top3
0,22899,9,6011,4010,50847.54,6050,16835,173911093540,4171603000
1,27914,4,4814,1030,74115.21,15060,16835,173911093540,50686506000
2,28753,13,4814,1010,-2589800.29,3150,15457,135156930084,1154733000
3,31385,13,4814,1030,-83525.38,3150,16236,156577858094,1154733000
4,38084,26,6011,1010,693495.66,9230,16251,157065740909,8566921000
...,...,...,...,...,...,...,...,...,...
8651,99967537,1,6011,2010,-336887.37,2010,6011,6011,2010
8652,99984336,4,6011,7010,78607.06,9020,6011,6011,14090100
8653,99985917,1,6011,2010,-224591.58,2010,6011,6011,2010
8654,99991245,4,6011,7010,569609.16,10350,16339,159620127381,16496633000


In [44]:
train_df_2 = train_set.merge(ml_df)
X_2 = train_df_2.drop(columns=['client_id', 'target'])
y_2 = train_df_2.target
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=.2, random_state=42)
cb_model_2 = CatBoostClassifier(verbose=100)
cb_model_2.fit(X_train_2, y_train_2)
y_pred_2 = cb_model_2.predict(X_test_2)
print('----------------------')
print('Accuracy score: %.2f\n'
      'Precision score: %.2f\n'
      'Recall score: %.2f\n'
      'f1 score: %.2f\n'
      'Roc_auc score: %.2f' % (accuracy_score(y_test_2, y_pred_2), precision_score(y_test_2, y_pred_2),
                               recall_score(y_test_2, y_pred_2), f1_score(y_test_2, y_pred_2),
                               roc_auc_score(y_test_2, y_pred_2)))

Learning rate set to 0.020129
0:	learn: 0.6922737	total: 2.54ms	remaining: 2.54s
100:	learn: 0.6567627	total: 194ms	remaining: 1.73s
200:	learn: 0.6424986	total: 406ms	remaining: 1.61s
300:	learn: 0.6315262	total: 606ms	remaining: 1.41s
400:	learn: 0.6212189	total: 838ms	remaining: 1.25s
500:	learn: 0.6100899	total: 1.07s	remaining: 1.07s
600:	learn: 0.5974037	total: 1.31s	remaining: 873ms
700:	learn: 0.5852065	total: 1.54s	remaining: 656ms
800:	learn: 0.5743341	total: 1.76s	remaining: 439ms
900:	learn: 0.5650250	total: 1.99s	remaining: 219ms
999:	learn: 0.5557663	total: 2.23s	remaining: 0us
----------------------
Accuracy score: 0.63
Precision score: 0.59
Recall score: 0.41
f1 score: 0.48
Roc_auc score: 0.60


In [45]:
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}
cb_model_grid = CatBoostClassifier(verbose=100)
grid_search_resul = cb_model_grid.grid_search(grid, 
                       X=X_train_2,
                       y=y_train_2)

0:	learn: 0.6922180	test: 0.6923653	best: 0.6923653 (0)	total: 9.5ms	remaining: 9.49s
100:	learn: 0.6598159	test: 0.6690648	best: 0.6690648 (100)	total: 201ms	remaining: 1.79s
200:	learn: 0.6469573	test: 0.6666579	best: 0.6664964 (194)	total: 427ms	remaining: 1.7s
300:	learn: 0.6369821	test: 0.6669996	best: 0.6661829 (268)	total: 655ms	remaining: 1.52s
400:	learn: 0.6242559	test: 0.6689674	best: 0.6661829 (268)	total: 927ms	remaining: 1.39s
500:	learn: 0.6121394	test: 0.6689367	best: 0.6661829 (268)	total: 1.18s	remaining: 1.17s
600:	learn: 0.6021586	test: 0.6713164	best: 0.6661829 (268)	total: 1.42s	remaining: 944ms
700:	learn: 0.5931976	test: 0.6727566	best: 0.6661829 (268)	total: 1.63s	remaining: 695ms
800:	learn: 0.5837134	test: 0.6750998	best: 0.6661829 (268)	total: 1.87s	remaining: 464ms
900:	learn: 0.5760500	test: 0.6759654	best: 0.6661829 (268)	total: 2.06s	remaining: 227ms
999:	learn: 0.5684853	test: 0.6763737	best: 0.6661829 (268)	total: 2.29s	remaining: 0us

bestTest = 0.666

600:	learn: 0.5260521	test: 0.6901023	best: 0.6676079 (70)	total: 1.42s	remaining: 946ms
700:	learn: 0.5123503	test: 0.6932021	best: 0.6676079 (70)	total: 1.64s	remaining: 700ms
800:	learn: 0.4994416	test: 0.6977259	best: 0.6676079 (70)	total: 1.84s	remaining: 458ms
900:	learn: 0.4873773	test: 0.7023732	best: 0.6676079 (70)	total: 2.1s	remaining: 231ms
999:	learn: 0.4764435	test: 0.7081178	best: 0.6676079 (70)	total: 2.38s	remaining: 0us

bestTest = 0.6676079294
bestIteration = 70

7:	loss: 0.6676079	best: 0.6661829 (0)	total: 17.5s	remaining: 1m 9s
0:	learn: 0.6922474	test: 0.6923651	best: 0.6923651 (0)	total: 3.21ms	remaining: 3.2s
100:	learn: 0.6611006	test: 0.6708471	best: 0.6708471 (100)	total: 238ms	remaining: 2.12s
200:	learn: 0.6507961	test: 0.6687507	best: 0.6687342 (199)	total: 459ms	remaining: 1.82s
300:	learn: 0.6417115	test: 0.6673307	best: 0.6669533 (274)	total: 742ms	remaining: 1.72s
400:	learn: 0.6320695	test: 0.6676762	best: 0.6669533 (274)	total: 1.01s	remaining: 1.51

100:	learn: 0.6068079	test: 0.6706991	best: 0.6678747 (51)	total: 263ms	remaining: 2.34s
200:	learn: 0.5471689	test: 0.6743415	best: 0.6678747 (51)	total: 536ms	remaining: 2.13s
300:	learn: 0.5064422	test: 0.6835615	best: 0.6678747 (51)	total: 818ms	remaining: 1.9s
400:	learn: 0.4714028	test: 0.6939684	best: 0.6678747 (51)	total: 1.11s	remaining: 1.66s
500:	learn: 0.4420912	test: 0.7055246	best: 0.6678747 (51)	total: 1.4s	remaining: 1.4s
600:	learn: 0.4163531	test: 0.7154314	best: 0.6678747 (51)	total: 1.69s	remaining: 1.12s
700:	learn: 0.3914173	test: 0.7234980	best: 0.6678747 (51)	total: 1.97s	remaining: 842ms
800:	learn: 0.3701191	test: 0.7345103	best: 0.6678747 (51)	total: 2.29s	remaining: 568ms
900:	learn: 0.3523512	test: 0.7423485	best: 0.6678747 (51)	total: 2.58s	remaining: 284ms
999:	learn: 0.3357787	test: 0.7531639	best: 0.6678747 (51)	total: 2.85s	remaining: 0us

bestTest = 0.6678747094
bestIteration = 51

15:	loss: 0.6678747	best: 0.6661829 (0)	total: 38.3s	remaining: 57.4s


800:	learn: 0.4320791	test: 0.6986686	best: 0.6705883 (108)	total: 4.65s	remaining: 1.16s
900:	learn: 0.4144922	test: 0.7034818	best: 0.6705883 (108)	total: 5.22s	remaining: 574ms
999:	learn: 0.3985833	test: 0.7066877	best: 0.6705883 (108)	total: 5.88s	remaining: 0us

bestTest = 0.6705883076
bestIteration = 108

22:	loss: 0.6705883	best: 0.6661829 (0)	total: 1m 7s	remaining: 49.8s
0:	learn: 0.6874285	test: 0.6908244	best: 0.6908244 (0)	total: 4.97ms	remaining: 4.97s
100:	learn: 0.5587452	test: 0.6824610	best: 0.6745717 (31)	total: 571ms	remaining: 5.08s
200:	learn: 0.4598942	test: 0.6946065	best: 0.6745717 (31)	total: 1.06s	remaining: 4.24s
300:	learn: 0.3908490	test: 0.7159683	best: 0.6745717 (31)	total: 1.8s	remaining: 4.17s
400:	learn: 0.3463022	test: 0.7296895	best: 0.6745717 (31)	total: 2.33s	remaining: 3.48s
500:	learn: 0.3082576	test: 0.7454069	best: 0.6745717 (31)	total: 2.87s	remaining: 2.85s
600:	learn: 0.2752614	test: 0.7649758	best: 0.6745717 (31)	total: 3.29s	remaining: 2.

200:	learn: 0.5151945	test: 0.6850406	best: 0.6764213 (54)	total: 2.57s	remaining: 10.2s
300:	learn: 0.4684443	test: 0.6953065	best: 0.6764213 (54)	total: 3.86s	remaining: 8.95s
400:	learn: 0.4151965	test: 0.7020599	best: 0.6764213 (54)	total: 5.21s	remaining: 7.78s
500:	learn: 0.3735743	test: 0.7083082	best: 0.6764213 (54)	total: 6.62s	remaining: 6.59s
600:	learn: 0.3361702	test: 0.7176360	best: 0.6764213 (54)	total: 8.03s	remaining: 5.33s
700:	learn: 0.3035086	test: 0.7247127	best: 0.6764213 (54)	total: 9.48s	remaining: 4.04s
800:	learn: 0.2767343	test: 0.7346197	best: 0.6764213 (54)	total: 10.8s	remaining: 2.67s
900:	learn: 0.2546864	test: 0.7446112	best: 0.6764213 (54)	total: 12s	remaining: 1.32s
999:	learn: 0.2345906	test: 0.7527721	best: 0.6764213 (54)	total: 13.3s	remaining: 0us

bestTest = 0.6764212879
bestIteration = 54

30:	loss: 0.6764213	best: 0.6661829 (0)	total: 1m 59s	remaining: 34.7s
0:	learn: 0.6856225	test: 0.6909128	best: 0.6909128 (0)	total: 12.8ms	remaining: 12.7s


900:	learn: 0.1644053	test: 0.8422940	best: 0.6749846 (18)	total: 12.3s	remaining: 1.35s
999:	learn: 0.1504657	test: 0.8592547	best: 0.6749846 (18)	total: 13.6s	remaining: 0us

bestTest = 0.6749845775
bestIteration = 18

37:	loss: 0.6749846	best: 0.6661829 (0)	total: 3m 34s	remaining: 11.3s
0:	learn: 0.6913298	test: 0.6923056	best: 0.6923056 (0)	total: 13.1ms	remaining: 13s
100:	learn: 0.6046480	test: 0.6743090	best: 0.6743090 (100)	total: 1.3s	remaining: 11.6s
200:	learn: 0.5674906	test: 0.6781292	best: 0.6743090 (100)	total: 2.6s	remaining: 10.4s
300:	learn: 0.5330785	test: 0.6841140	best: 0.6743090 (100)	total: 3.93s	remaining: 9.12s
400:	learn: 0.5004433	test: 0.6865072	best: 0.6743090 (100)	total: 5.21s	remaining: 7.79s
500:	learn: 0.4706845	test: 0.6909426	best: 0.6743090 (100)	total: 6.5s	remaining: 6.47s
600:	learn: 0.4455295	test: 0.6962738	best: 0.6743090 (100)	total: 7.78s	remaining: 5.16s
700:	learn: 0.4213492	test: 0.7018490	best: 0.6743090 (100)	total: 9.19s	remaining: 3.

In [46]:
grid_search_resul

{'params': {'depth': 4, 'l2_leaf_reg': 1, 'learning_rate': 0.03},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
    

In [47]:
final_model = CatBoostClassifier(depth= 4,
                                l2_leaf_reg= 1,
                                learning_rate= 0.03,
                                verbose=100)

In [48]:
final_model.fit(X_train_2, y_train_2)
y_pred_final = final_model.predict(X_test_2)
print('----------------------')
print('Accuracy score: %.2f\n'
      'Precision score: %.2f\n'
      'Recall score: %.2f\n'
      'f1 score: %.2f\n'
      'Roc_auc score: %.2f' % (accuracy_score(y_test_2, y_pred_final), precision_score(y_test_2, y_pred_final),
                               recall_score(y_test_2, y_pred_final), f1_score(y_test_2, y_pred_final),
                               roc_auc_score(y_test_2, y_pred_final)))

0:	learn: 0.6921865	total: 1.46ms	remaining: 1.46s
100:	learn: 0.6606074	total: 153ms	remaining: 1.36s
200:	learn: 0.6502378	total: 305ms	remaining: 1.21s
300:	learn: 0.6417686	total: 451ms	remaining: 1.05s
400:	learn: 0.6303875	total: 598ms	remaining: 894ms
500:	learn: 0.6206407	total: 741ms	remaining: 738ms
600:	learn: 0.6113827	total: 894ms	remaining: 594ms
700:	learn: 0.6038880	total: 1.04s	remaining: 444ms
800:	learn: 0.5961795	total: 1.21s	remaining: 302ms
900:	learn: 0.5890899	total: 1.38s	remaining: 151ms
999:	learn: 0.5823122	total: 1.53s	remaining: 0us
----------------------
Accuracy score: 0.64
Precision score: 0.61
Recall score: 0.41
f1 score: 0.49
Roc_auc score: 0.61


# Prediction

In [49]:
test_df = test_set.merge(ml_df)
test_df.head()

,client_id,transactions_count,code_mode,type_mode,sum,sum_type,sum_code,mult_code_top3,mult_type_top3
0,61240380,15,5411,1010,-245029.43,3150,15766,144335037714,1154733000
1,34114030,3,6011,1010,170464.33,10030,11510,33054489,14231001000
2,15926722,3,6011,2010,-406510.75,2010,6011,6011,2010
3,50598019,31,5921,1010,-102308.88,4130,17343,192583609841,2253411000
4,76475250,37,5411,1010,-5836986.22,4130,16753,173393552451,2253411000


In [50]:
y_test_pred = final_model.predict(test_df)

In [51]:
answer = test_set.copy()

In [52]:
answer['target'] = y_test_pred

In [53]:
answer

,client_id,target
0,61240380,0
1,34114030,0
2,15926722,1
3,50598019,1
4,76475250,1
...,...,...
2651,13491189,1
2652,54047312,1
2653,68859452,1
2654,95721840,0


In [57]:
answer.to_csv('C:/Users/Mr0Wo/OneDrive/Рабочий стол/Git_repositories/Data-science-cases/ML-cases/user-gender-detection/solution.csv', index=False)